In [1]:
import json

import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry
from vo_colos.utils.object_registry import ClassInspect

from vo_colos.utils.base_task import SimulationFactory

In [2]:
with open("./config hopper 2.json", 'r') as fp:
    config = json.load(fp)

In [3]:
class message:
    id = 1
    task = {
        "id": 1,
        "engine": "colos",
        "project_id": "1234567890",
        "tag": "testsimulation",
        "config": config["config"],
    }
    status = "new"

sim = SimulationFactory(message)

In [4]:
sim.simulate()

init
level: 55
completed init
init
level: 55
completed init
init
level: 55
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
while Activity keep_resources []
start put_available
register new event 
start event instance None
while Activity keep_resources []
start put_available
register new event 
start event instance None
while Activity keep_resources []
start put_available
register new event 
start event expression [{'and': [{'type': 'activity', 'name': 'SingeRunCycle_1', 'state': 'done'}, {'type': 'activity', 'name': 'SingeRunCycle_3', 'state': 'done'}]}]
start event <AllOf(all_events, (<Process(conditional_process) object at 0x1aa22eaa5c8>, <Process(conditional_process) object at 0x1aa22eac988>)) object at 0x1aa22eaff48>
start event instance <AllOf(all_events, (<Process(conditional_process) object at 0x1aa22eaa5c8>, <Process(conditional_process) object at 0x1aa22eac988>)) object at 0x1aa22eaff48>
conditi

released processor : {}
{}
Activity end(loading_1)
keep_resources []
start event instance None
Mover_move before mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end(sailing_full_1)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.1734f44c-abc2-11ea-bda8-2079183a5c9a': 5, 'destination.to_site': 999975}
destination store before put: [{'id': 'default', 'level': 25, 'capacity': 1000000}]
<FilterStoreGet() object at 0x1aa22e9d608>
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 30, 'capacity': 1000000}]
after put
processor end process
Processed 5 of default:
  by:          boot_3
  origin        boot_3  contains: 0 of default
  destination:  to_site contains: 30 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x000001AA19711F88>: <Request() object at 0x1aa22eb4108>, <simpy.resources.resource.Resource object at 0x000001AA22E

start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ecb248>}
site request : {<simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ecb248>, <simpy.resources.resource.Resource object at 0x000001AA1E921E88>: <Request() object at 0x1aa22a2a3c8>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ecb248>, <simpy.resources.resource.Resource object at 0x000001AA1E921E88>: <Request() object at 0x1aa22a2a3c8>}
after req resource if available : {<simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ecb248>, <simpy.resources.resource.Resource object at 0x000001AA1E921E88>: <Request() object at 0x1aa22a2a3c8>}
_shift_amount
start _shift_amount
amount 5
processor start process
transfer default to boot_3
processor process without rate
origin store before get: [{'id': 'de

{}
origin store after get: [{'id': 'default', 'level': 0, 'capacity': 5}]
after get
after check_down time
destination store before put: [{'id': 'default', 'level': 95, 'capacity': 1000000}]
<FilterStoreGet() object at 0x1aa1972a808>
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 100, 'capacity': 1000000}]
after put
processor end process
Processed 5 of default:
  by:          boot_3
  origin        boot_3  contains: 0 of default
  destination:  to_site contains: 100 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x000001AA19711F88>: <Request() object at 0x1aa22ed2908>, <simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ee3608>}
keep resources []
release destination : {<simpy.resources.resource.Resource object at 0x000001AA22E9DE88>: <Request() object at 0x1aa22ee3608>}
released origin : {}
released processor : {}
{}
Activity end(unloading_3)
Activity end(WhileFull_3)
conditi

start _shift_amount
amount 5
processor start process
transfer default to to_site
processor process without rate
origin store before get: [{'id': 'default', 'level': 5, 'capacity': 5}]
start get 5
store_status {'id': 'default', 'level': 5, 'capacity': 5}
end get 5
get_callback - id_ <StorePut() object at 0x1aa22ea68c8>
start get_callback
{}
origin store after get: [{'id': 'default', 'level': 0, 'capacity': 5}]
after get
after check_down time
destination store before put: [{'id': 'default', 'level': 130, 'capacity': 1000000}]
<FilterStoreGet() object at 0x1aa22eaa3c8>
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 135, 'capacity': 1000000}]
after put
processor end process
Processed 5 of default:
  by:          boot_1
  origin        boot_1  contains: 0 of default
  destination:  to_site contains: 135 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x000001AA19711F88>: <Request() object at 0x1aa22ed23c8>, <simpy.resourc

In [5]:
objects = list(sim.vessels)
objects.extend(sim.locations)


activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

vessel_planning(objects, activities)